# Fazer Upload planilhas
1.   Mintel
2.   Inci Produtos WACKER
3.   Vendas Distribuidor
4.   Projetos Distribuidor

In [1]:
#Mintel
from google.colab import files
uploaded = files.upload()

Saving Q1-2023-Cosmeticos.xlsx to Q1-2023-Cosmeticos.xlsx


In [2]:
filename = next(iter(uploaded))

In [3]:
# INCI name produtos WACKER
from google.colab import files
inci = files.upload()

Saving inci.xlsx to inci.xlsx


In [4]:
filename2 = next(iter(inci))

In [5]:
#Projetos Distribuidores
from google.colab import files
proj = files.upload()

Saving Projetos.xlsx to Projetos.xlsx


In [6]:
filename3 = next(iter(proj))

In [7]:
#Vendas Distribuidores
from google.colab import files
dist = files.upload()

Saving Vendas.xlsx to Vendas.xlsx


In [8]:
filename4 = next(iter(dist))

# Bibliotecas Phyton

In [9]:
#Carrega Bibliotecas
import pandas as pd
import numpy as np
import re

In [10]:
pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 14.4 MB/s eta 0:00:00


In [11]:
from unidecode import unidecode

##Carrega tabela do Mintel

In [12]:
#Carrega o banco de dados como tabela
data = '/content/'+filename
df_mintel = pd.read_excel(data)
pd.options.display.max_columns = None
df_mintel.nunique()

Número do Produto               1136
Data de Publicação                64
Produto                          807
Marca                            811
Empresa                          347
Mercado                            1
Categoria                          4
Sub-Categoria                     36
Descrição do Produto            1136
Tipo de Lançamento                 5
Fabricante                       330
Local de fabricação               20
Manufacturer Company Address     339
Ingredients (Standard form)     1112
Preço em moeda local             510
Link da Imagem Primária         1136
Animal de Estimação                0
Conveniência                      20
Menos                              0
Funcional                         58
Natural                           20
Mais                               2
Posicionamento                    10
Adequado para                      4
Demográfico                       10
Sem                               41
Realce de Beleza                  90
É

In [13]:
#remove lançamentos Nova Embalagem
#df_mintel = df_mintel[df_mintel['Tipo de Lançamento']!='Nova Embalagem']

##Carrega Tabela de INCI Wacker

In [14]:
data = '/content/'+filename2
df_inci = pd.read_excel(data)
pd.options.display.max_columns=None
df_inci.nunique()

Produto        50
Ingrediente    46
Prioridade      3
dtype: int64

##Carrega Tabela de Projetos Wacker

In [15]:
data = '/content/'+filename3
df_proj = pd.read_excel(data)
pd.options.display.max_columns=None
df_proj.nunique()

Distribuidor      3
Customer        913
UF               16
Product         144
Fase             17
dtype: int64

## Carrega Planilha de Vendas do Distribuidor

In [16]:
data = '/content/'+filename4
df_dist = pd.read_excel(data)
pd.options.display.max_columns=None
df_dist.nunique()

Customer(at distrib)        891
Country, Customer             1
Unnamed: 2                    1
Distributor                   4
Product management            3
Material (at WAC)            55
Unnamed: 6                   54
Market segment                1
Unnamed: 8                    1
Application                   6
Month last sale|Timeline     12
2022                        346
dtype: int64

# Define funções

In [17]:
#Cruza Mintel e INCI
def verifica_ingrediente(formula,produto,material):
  quantidade = len(produto.difference(formula))
  if quantidade == 0:
    return material
  return None

In [18]:
#Retorna possíveis produtos Baseado em Mintel
def procura_produtos(formula, produtos, materiais):
  formula = formula.copy()
  resultados = []
  for prod, mat in zip(produtos, materiais):
    resultado = verifica_ingrediente(formula, prod, mat)
    if resultado is not None:
      formula = formula.difference(prod) ## Para remover os ingredientes já encontrados numa nova busca.
      resultados.append(resultado)
  return resultados

In [19]:
#Transpor
def transpor (df, coluna, linha):
  for cat in df[coluna].unique():
    f = df[coluna] == cat
    df[cat] = df[f][linha]
    f = df[cat].isna()  
    df.loc[f, cat] = df.loc[f, cat].apply(lambda x:[])

In [20]:
#Resolver problemas de set
def to_set(x):
    try:
        if isinstance(x, pd.DataFrame):
            return x.apply(to_set)
        else:
            return set(x.sum())
    except:
        pass

In [21]:
#Resolver problemas de set
def to_set_2(x):
    try:
        if isinstance(x, pd.DataFrame):
            return x.apply(to_set_2)
        else:
            return list(set(x))
    except:
        pass

In [22]:
def formatar_fabricantes(coluna):
  remover = [
             'fabrica', 'fabricacao', 'manuf', 'tercerizacao', 'manufacturing',
             'farmacia', 'farmacias', 'farmaceuticos', 'drogaria','drogarias', 'farm',
             'cosmetico', 'cometico','cosmeticos', 'cometicos', 'cosmetics',
             'cosmiatria', 'cosmetica', 'cosmeticas', 'cosm', 'cosme', 'kosmetics',
             'ltda','epp', 'eireli', 'me', 'eirell', 'sa', 'eir', 'ltd',
             'indústria','industria', 'industrias', 'industrial', 'indutrial', 'ind', 'industriae',
             'comercio', 'comecio', 'comercial', 'corporation', 'com', 
             'imp', 'exp',  'prod', 'exportacao','importacao','importadora', 'import',
             'do', 'e', 'para', 'de', 'le', 'la',
             'do brasil', 'aerossois', 'aerosol', 'aerossol', 'tecnologia', 
             'laboratorio','laboratorios', 'laboratories', 'laboratoire', 'instituto', 'group', 
             'pesquisa', 'desenvolvimento', 'naturais', 'professional', 'aromatizantes',
             'produtos', 'produto', 'saneantes', 'quimica', 'quimicos', 'quimico', 'farmaceutica', 'veterinário',
             'higiene','pessoal', 'limpeza', 'beleza', 'botanico',
             'perfumes','fragrancias', 'perfumaria', 'medicamentos', 'perf',
             '\.', "\'", '\d+', #Remove números
             'aw', 'a w', 'a  w', 'distribuidora', 'dist', 
             'international', 'internacional', 'sulamericana', 'nacional',
             'nbsp;', 'veer','serv', 'vajerista', 
           ]
  pat = '\\b({})\\b'.format('|'.join(remover))
  coluna = (
    coluna
    .apply(unidecode)
    .str.lower()
    .replace('\.', ' ',regex=True)
    .replace('\&', ' ',regex=True)
    .replace('\/', '',regex=True)
    .replace('\-', '',regex=True) 
    .replace(pat,'',regex=True)
    .replace(',', '',regex=True) 
    .replace('\s+','',regex=True) #Tirar numeros
    .str.strip()
  )
  return coluna

In [23]:
def formatar_material(coluna):
  remover = ['BELSIL®', 'BELSIL', 'BELSIM',
             'DRUM', 'ST', 'PE', 'CANISTER', 'RD', 'CARTON', 'IBC PE', '2', 'WOPAL', 
             'SBPAL', 'STPAL', 'HOBBOCK', 'SACK', 'BOTTLE',
             '0,45KG','10KG','15KG','25KG', '30G', '60G', '100KG', '140KG', '150KG', '160KG',
             '180KG', '190KG', '200KG', '210KG', '900KG', '950KG', '190', 
             'KG', 'AMOSTRA', 'PA', 'XY', 'CP', 'SILICONE ELASTOMER GEL',
             'BR', 'DE', ' E', 'BLOCKED',
             'EMB. 0,100 KG', 'Ver  Pf 100', 'USP',
           ]
  pat = '\\b({})\\b'.format('|'.join(remover))
  coluna = (
    coluna
    .replace(pat,'',regex=True)
    .replace('\#','', regex=True)
    .replace('\"','', regex=True)
    .str.replace('\/', '',regex=True)
    .str.replace('\-', '',regex=True)
    .str.replace('\*', '',regex=True)      
    .str.title()
    .str.strip()
  )
  return coluna

# Mintel

## Define Estados


In [24]:
dict = {'AC':'AC', 'AL':'AL', 'AM':'AM', 'AP':'AP', 'BA':'BA', 'CE':'CE', 'DF':'DF', 'ES':'ES', 'GO':'GO', 'MA':'MA', 
     'MG':'MG', 'MS':'MS', 'MT':'MT', 'PA':'PA', 'PB':'PB', 'PE':'PE', 'PI':'PI', 'PR':'PR', 'RJ':'RJ', 'RN':'RN',
     'RO':'RO', 'RR':'RR', 'RS':'RS', 'SC':'SC', 'SE':'SE', 'SP':'SP', 'TO':'TO', 'Sao Paulo':'SP', 'São Paulo':'SP',
     'Sao Paolo':'SP','Osasco': 'SP', 'Mogi das Cruzes': 'SP', 'Cotia':'SP', 'Diadema':'SP', 'São Roque':'SP', 
     'São Sebastião':'SP', 'Lins':'SP', 'Cajamar':'SP', 'Pirassununga':'SP', 'Paulista': 'SP', 'São Paolo':'SP', 
     'Elisario':'SP', 'Valinhos':'SP', 'Piracaia':'SP', 'São José do Rio Preto':'SP', 'Sao José do Rio Preto':'SP',
     'São José dos Campos':'SP',      'Araras':'SP', 'São Bernardo do Campo':'SP', 'São José dos Campos':'SP',
     'Sao Caetano do Sul':'SP', 'São José do Rio Preto':'SP', 'Brodowski':'SP', 'Cajuru':'SP', 'Itaquaquecetuba':'SP', 
     'Itupeva':'SP', 'SO':'SP', 'Mairiporã':'SP', 'Itaquera':'SP', 'Franco da Rocha':'SP', 'Guarulhos':'SP',
     'Bairro Jardim America':'SP', 'Jardim Brazil':'SP', 'Agudos':'SP', 'Itapira':'SP', 'Rib. Preto':'SP',
     'Rio de Janeiro':'RJ', 'Rio De Janeiro':'RJ', 'Nova Iguaçu': 'RJ', 'Nova Iguacu': 'RJ', 'Nivopolis':'RJ',
     'Duque de Caixas':'RJ', 'Engenheiro Pedreira':'RJ',
     'Vila Velha':'ES', 'Marechal Floriano':'ES', 
     'Belo Horizonte':'MG', 'Minas Gerais':'MG', 'Pocos de Caldas':'MG', 'Viçosa':'MG', 'Vicosa':'MG',
     'Pouso Alegre': 'MG', 'Uberlândia':'MG', 'Uberlandia':'MG', 'Ipatinga':'MG', 'Mirabela':'MG','Nova Lima':'MG',
     'São José da Lapa':'MG', 'Sao José da Lapa':'MG', 'Sao Jose da Lapa':'MG',
     'Porto Alegre':'RS', 'Estância Velha': 'RS', 'Caxias do Sul':'RS', 'Passo Fundo':'RS', 'Gravatai':'RS',
     'Santa Catarina':'SC', 'Joinville':'SC','Camboriu':'SC', 'Maluche':'SC', 'Araquari':'SC', 'Criciúma': 'SC',
     'Criciuma': 'SC',
     'Colombo':'PR', 'Paraná':'PR', 'Curitiba':'PR', 'Sao José dos Pinhais':'PR','BR 116 Km 85':'PR', 'Cambé':'PR',
     'Goiana':'GO', 'Goiânia':'GO', 'Anápolis':'GO', 'Anapolis':'GO', 'Brasilia': 'DF', 'Go':'GO',
     'Rondônia':'RO','Novo Hamburgo':'RO','Manaus':'AM', 'Pará':'PA',
     'Bahia':'BA', 'Lauro de Freites': 'BA', 'Av. Queira Deus':'BA', 'Vila Cia Sul':'BA',
     'Recife':'PE', 'Camaragibe': 'PE', 'Fortaleza':'CE', 'Ceará':'CE'}

In [25]:
df_mintel['Estados'] = df_mintel['Manufacturer Company Address'].str.extract('('+'|'.join(dict)+')',expand=False).map(dict)
#Exibe a quantidade de casos únicos
df_mintel['Estados'].value_counts(dropna=False)
#Em output se filtra estados vazios

SP     574
NaN    220
MG      69
RJ      60
GO      56
PR      48
RS      40
SC      16
CE      14
BA      13
PE      11
ES       6
PI       3
DF       2
SE       2
AM       1
PA       1
Name: Estados, dtype: int64

## Remover Clientes WACKER da planilha Mintel


In [26]:
dictClientes = {'Devintex Cosméticos':'1', 'Devintex':'1','Procosa Produtos de Beleza':'1', 'Colgate-Palmolive':'1', 'Unilever':'1',
                'Savoy Indústria de Cosméticos':'1', 'Beiersdorf':'1', 'Johnson & Johnson':'1',
                'Procter & Gamble':'1','Química Amparo':'1', "Oréal":'1', 'Procosa':'1',
                'Unilever Brasil Industrial':'1', 'Botica Comercial Farmacêutica':'1'}
dictMarcas = { 'Boticário':'1', 'Siàge':'1', 'Natura ':'1', 'Avon ':'1', 'Eudora':'1', 
              'Wella':'1', 'The Body Shop':'1', 'Quem Disse,':'1', 'Salon Line':'1', 'Maybelline':'1'}
df_mintel['Clientes']= df_mintel['Fabricante'].str.extract('('+'|'.join(dictClientes)+')',expand=False).map(dictClientes)
df_mintel['Clientes2'] = df_mintel['Marca'].str.extract('('+'|'.join(dictMarcas)+')',expand=False).map(dictMarcas)
df_wacker = df_mintel[df_mintel['Clientes']=='1']
df_wacker = df_wacker[df_wacker['Clientes2']=='1']
df_wacker.to_excel("LancClientesWACKER.xlsx") 

In [27]:
#Remove clientes Wacker da base de tratamento
df_mintel = df_mintel[df_mintel['Clientes']!='1']
df_mintel = df_mintel[df_mintel['Clientes2']!='1']

## Define distribuidor por território

In [28]:
dict_terr = {'PR':'Embacaps', 'RS':'Embacaps', 'SC':'Embacaps',
            'BA':'Morais', 'PE':'Morais', 'CE':'Morais',
            'SP':'Focus', 'RJ': 'Focus', 'MG':'Focus', 'GO':'Focus', 'ES':'Focus',
}
df_mintel['Territorio'] = df_mintel['Estados'].str.extract('('+'|'.join(dict_terr)+')',expand=False).map(dict_terr)

## Tem Silicone

In [29]:
#Dicionário Silicones geral
dictOTHERS = {'methicone':'1', 'Dimethicone':'1','methicone Crosspolymer':'1','methiconol':'1',
              'ylsiloxysilicate':'1', 'ylsilsesquioxane':'1', 'Disiloxane':'1', 'Silica':'1','siloxane':'1'}

In [30]:
df_mintel['Silicone'] = df_mintel['Ingredients (Standard form)'].str.extract('('+'|'.join(dictOTHERS)+')',expand=False).map(dictOTHERS)

##Tem Claim Sem Silicone

In [31]:
dictSil = {'Silicone':'1'}

In [32]:
df_mintel['Claim Sem Silicone'] = df_mintel['Sem'].str.extract('('+'|'.join(dictSil)+')',expand=False).map(dictSil)

## Busca em Ingredientes no Mintel e retorna produto WACKER



In [33]:
df_inci.dropna(inplace=True)
df_inci['Produto'] = formatar_material(df_inci['Produto'])

In [34]:
#Cria uma lista iterável dos ingredientes nos Produtos Wacker
df_inci['Ing'] = formatar_material(df_inci['Ingrediente']).str.split(', ').apply(set)

In [35]:
#Conta a quantidade de ingredientes
df_inci.sort_values('Prioridade', ascending=True, inplace=True)

In [36]:
 #Dicionário de palavras a remover da coluna Ingredientes no Mintel
 dictIng={' Andor ':',',
          ', ':',',
          ' ,':',',
          ' (And) ':',',
          }

In [37]:
#Cria uma lista iterável das formulas do mintel 
df_mintel['Ingredients (Standard form)'] = df_mintel['Ingredients (Standard form)'].astype(str)
df_mintel['Ing'] = formatar_material(df_mintel['Ingredients (Standard form)']).replace(dictIng,regex=True).dropna().str.split(',').apply(set)

In [38]:
#Aplica a função Procura Produtos
df_mintel['WACKER'] = df_mintel['Ing'].apply(lambda formulacao:procura_produtos(formulacao, df_inci['Ing'], df_inci['Produto']))

## Cria Indice para junção

In [39]:
df_mintel.set_index(['Número do Produto'])
df_mintel['Data de Publicação'] = pd.to_datetime(df_mintel['Data de Publicação'], format='%Y-%m-%d')
df_mintel['Trimestre'] = df_mintel['Data de Publicação'].dt.quarter
df_mintel['Ano'] = df_mintel['Data de Publicação'].dt.year

In [40]:
#Cria a coluna que irá indexar a mistura com a planilha do mintel
df_mintel['Fabricante']=df_mintel['Fabricante'].astype(str)
df_mintel['Cliente Resumo'] = formatar_fabricantes(df_mintel['Fabricante'])

In [41]:
def join(x):   
    r = ' '.join(x.dropna().iloc[:2])
    return r
#Coloca as duas primeiras palavra da marca
df_mintel['Marca']=df_mintel['Marca'].astype(str)
df_mintel['Marca Resumo'] = df_mintel['Marca'].str.replace('La','', regex=True).str.split(expand=True).apply(join, axis=1)
#df_mintel['Marca Resumo'] = formatar_fabricantes(df_mintel['Marca Resumo'])

In [42]:
df_mintel['Marca Resumo']

0                   Derma Chem
2                    Ruby Rose
3       Phállebeauty Cosmetics
11            Darrow boratório
12               Naos Bioderma
                 ...          
1131            Face Beautiful
1132               Griffus Vou
1133          PanVel Instantes
1134          PanVel Instantes
1135          PanVel Instantes
Name: Marca Resumo, Length: 912, dtype: object

## Baixa relatório Linha a Linha, com analise de ingrediente

In [43]:
df_mintel.columns

Index(['Número do Produto', 'Data de Publicação', 'Produto', 'Marca',
       'Empresa', 'Mercado', 'Categoria', 'Sub-Categoria',
       'Descrição do Produto', 'Tipo de Lançamento', 'Fabricante',
       'Local de fabricação', 'Manufacturer Company Address',
       'Ingredients (Standard form)', 'Preço em moeda local',
       'Link da Imagem Primária', 'Animal de Estimação', 'Conveniência',
       'Menos', 'Funcional', 'Natural', 'Mais', 'Posicionamento',
       'Adequado para', 'Demográfico', 'Sem', 'Realce de Beleza',
       'Ético e ambiental', 'Produto testado', 'Estados', 'Clientes',
       'Clientes2', 'Territorio', 'Silicone', 'Claim Sem Silicone', 'Ing',
       'WACKER', 'Trimestre', 'Ano', 'Cliente Resumo', 'Marca Resumo'],
      dtype='object')

In [44]:
col=['Trimestre','Sub-Categoria', 'Produto', 'Marca', 'Empresa', 'Fabricante', 'Estados', 'Sem', 'Preço em moeda local', 
     'Silicone', 'WACKER', 'Ing', 'Manufacturer Company Address', 'Link da Imagem Primária','Territorio']

In [45]:
df_mintel[col].to_excel("Lancamentos Linha a Linha.xlsx")  

## Baixar relatório agrupar por Fabricante

In [46]:
transpor(df_mintel, 'Categoria', 'WACKER')

In [47]:
df_mintel.rename(columns={'Produtos para Pele':'Pele', 'Produtos para Cabelos':'Cabelo',	
                          'Sabonetes e Produtos para Banho':'Banho', 'Número do Produto': 'Total Lançamentos'}, 
                 inplace=True)

In [48]:
df_a = df_mintel.groupby(['Cliente Resumo', 'Trimestre', 'Estados'])[['Cabelo', 'Pele', 'Maquilagem', 'Banho']].apply(to_set)

In [49]:
df_b = df_mintel.groupby(['Cliente Resumo', 'Trimestre', 'Estados']).agg({
                                            'Total Lançamentos': 'count',
                                             'Silicone':'count',
                                             'Claim Sem Silicone':'count',
                                             })

In [50]:
df_mintel_fab = pd.concat([df_a, df_b], axis=1).reset_index()

In [51]:
#df_mintel_fab.to_excel('Mintel Agrupado Fabricantes.xlsx')

## Filtro de Datas

In [52]:
#df_mintel_fab = df_mintel_fab[df_mintel_fab['Ano'].isin([2022, 2021, 2020])]

# Vendas

In [53]:
#Garantir que apenas serão considerados os clientes com vendas
df_dist['2022'].dropna(inplace=True)

In [54]:
df_dist['Unnamed: 6']= formatar_material(df_dist['Unnamed: 6'])
df_dist['Unnamed: 6'] = df_dist['Unnamed: 6'].astype(str)

In [55]:
df_dist['Cliente Resumo'] = formatar_fabricantes(df_dist['Customer(at distrib)'])

In [56]:
#agrupo os produtos em uma lista, por cliente
df_dist_unic = df_dist.groupby(['Cliente Resumo','Distributor']).agg({'2022':'sum','Unnamed: 6':lambda x:list(set(x))}).reset_index()

In [57]:
df_dist_unic['Unnamed: 6'] = df_dist_unic['Unnamed: 6'].apply(set)
df_dist_unic.rename(columns={'Unnamed: 6': 'Sales'}, inplace=True)

In [58]:
df_dist_unic.drop_duplicates(subset=['Cliente Resumo', 'Distributor'], keep='first', inplace=True)
df_dist_unic.rename(columns={'Distributor':'Distribuidor'}, inplace=True)

In [59]:
df_dist_unic

,Cliente Resumo,Distribuidor,2022,Sales
0,,FOCUS QUIMICA,5.1724,"{Gb 150, Adm 9000, Adm 8105}"
1,,MORAIS DE CASTRO,0.4770,"{Gb 3020, Dm 6010, Ow 2100, Gb 3021}"
2,2k,EMBACAPS,0.0050,{Eg 5}
3,3x,BRENNTAG,0.3800,{Dm 1000}
4,abalo,FOCUS QUIMICA,0.2550,"{Cm 1306, Dm 6010, Ow 2100, Dm 350, Cm 1000, A..."
...,...,...,...,...
857,yantra,FOCUS QUIMICA,0.0300,"{Dm 350, Eg 5}"
858,yur,FOCUS QUIMICA,0.0300,"{Dm 0.65, Tms 803}"
859,yuzi,FOCUS QUIMICA,0.2130,"{Cm 1000, Dm 350, Tms 803, Eg 5}"
860,zaninpharma,EMBACAPS,0.1900,{Dm 350}


# Projetos

In [60]:
df_proj['Product']= formatar_material(df_proj['Product'])

In [61]:
df_proj['Cliente Resumo'] = formatar_fabricantes(df_proj['Customer'])

In [62]:
#Dicionário Silicones geral
dictFASE = {'APROVADO':'APROVADO','Closed Won': 'APROVADO', 'FINALIZADO': 'APROVADO',
            'CANCELADO':'CANCELADO', 'CANCELADO/REPROVADO': 'CANCELADO', 'STAND BY':'CANCELADO',
            'REPROVADO':'REPROVADO', 'Closed Lost': 'REPROVADO', 
            'EM ANDAMENTO':'EM ANDAMENTO', 'EM TESTE': 'EM ANDAMENTO', 'Negotiation/Review': 'EM ANDAMENTO',
            'Proposal/Price Quote':'EM ANDAMENTO', 'Prospecting':'EM ANDAMENTO',
            'Qualification':'EM ANDAMENTO',
            }

In [63]:
df_proj['FASE'] = df_proj['Fase'].str.strip().str.extract('('+'|'.join(dictFASE)+')',expand=False).map(dictFASE)

In [64]:
df_proj_unic = df_proj.groupby(['Cliente Resumo','UF','Distribuidor','FASE']).agg({
                                                          'Product': lambda x:list(set(x)),
                                                          }).reset_index()

In [65]:
transpor(df_proj_unic, 'FASE', 'Product')

In [66]:
df_proj_unic_2 = df_proj_unic.groupby(['Cliente Resumo', 'UF', 'Distribuidor'])[['CANCELADO', 'EM ANDAMENTO', 'APROVADO', 'REPROVADO']].apply(to_set).reset_index()

In [67]:
df_proj_unic_2.head(2)#.to_excel('projetosagrupados.xlsx')

,Cliente Resumo,UF,Distribuidor,CANCELADO,EM ANDAMENTO,APROVADO,REPROVADO
0,2k,SC,EMBACAPS,{Adm 8301},{},{},{}
1,a3eventosservicos,SC,EMBACAPS,{},{Eg 5},{},{}


# Agrupar Projeto e Vendas

In [68]:
df_proj_vend = df_proj_unic_2.merge(df_dist_unic, on=['Cliente Resumo', 'Distribuidor'], how='left')

In [69]:
df_proj_vend.to_excel('Projetos Vendas.xlsx')

# Agrupar Vendas e Projetos

In [70]:
df_vend_proj = df_dist_unic.merge(df_proj_unic_2, on=['Cliente Resumo', 'Distribuidor'], how='left')

In [71]:
#df_vend_proj.to_excel('Vendas Projetos.xlsx')

# Agrupar Mintel e Vendas e Projeto

In [72]:
df_mintel_vend_proj = df_mintel_fab.merge(df_vend_proj, on='Cliente Resumo', how='left')

In [73]:
df_mintel_vend_proj.rename(columns={'WACKER': 'WACKER Prod Poss','Customer(at distrib)':'Customer Sales','Distributor':'Dist. Sales', 
                                    'Sales': 'Prod Sales','2022': 'Sales Volume', 'Customer': 'Customer Proj Fab',
                                    'Distribuidor':'Dist Proj Fab', 'Project':'Prod Proj Fab', 
                                    'Fase':'Fase Proj Fab',
                                         }, 
                                inplace=True)
#df_mintel_vend_proj.drop_duplicates()
df_mintel_vend_proj.sample(2)

,Cliente Resumo,Trimestre,Estados,Cabelo,Pele,Maquilagem,Banho,Total Lançamentos,Silicone,Claim Sem Silicone,Dist Proj Fab,Sales Volume,Prod Sales,UF,CANCELADO,EM ANDAMENTO,APROVADO,REPROVADO
110,gcferreira,1,SP,NaN,NaN,NaN,NaN,2,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191,naturelle,1,SP,{Cm 740},"{Elastomero, Gb 150, Hdk N20, Cm 740}",{},{},10,9,0,FOCUS QUIMICA,0.785,"{Adm 8105, Dm 350, Pdm 20, Cm 1000, Adm 9000}",SP,{},{Gb 7030},{Eg 5},{}


In [74]:
df_mintel_vend_proj.to_excel('Mintel Vendas Projeto.xlsx')